In [ ]:
import psycopg2
from psycopg2 import sql
from openai import AzureOpenAI
import json
import os

In [ ]:

# Function to connect to PostgreSQL and fetch competency indicators
def get_competency_indicators(competency_ids):
    """
    Query PostgreSQL to fetch indicators grouped by levels for the given competencies.
    
    :param competency_ids: List of competency IDs to fetch indicators for.
    :return: Dictionary containing competencies, levels, and indicators.
    """
    try:
        # Connect to PostgreSQL database
        conn = psycopg2.connect(
            host=os.environ.get("POSTGRES_HOST"),
            database=os.environ.get("POSTGRES_DB"),
            user=os.environ.get("POSTGRES_USER"),
            password=os.environ.get("POSTGRES_PASSWORD")
        )
        cur = conn.cursor()

        # Prepare SQL query to fetch indicators for specific competencies
        query = sql.SQL("""
            SELECT competency_id, level, indicator
            FROM public.competency_indicators
            WHERE competency_id = ANY(%s)
            ORDER BY competency_id, level;
        """)

        # Execute query
        cur.execute(query, (competency_ids,))
        rows = cur.fetchall()

        # Organize data into a structured format (grouped by competency and level)
        competency_data = {}
        for row in rows:
            competency_id, level, indicator = row
            if competency_id not in competency_data:
                competency_data[competency_id] = {}
            if level not in competency_data[competency_id]:
                competency_data[competency_id][level] = []
            competency_data[competency_id][level].append(indicator)

        # Close the cursor and connection
        cur.close()
        conn.close()

        return competency_data

    except Exception as e:
        print(f"Error querying PostgreSQL: {e}")
        return None


In [ ]:
import os
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI

# --- Define the Pydantic model for ranking validation ---
class RankedIndicator(BaseModel):
    indicator: str = Field(..., description="The indicator being ranked.")
    rank: int = Field(..., description="The rank of the indicator.")

class RankedIndicatorsResponse(BaseModel):
    ranked_indicators: list[RankedIndicator] = Field(..., description="List of ranked indicators.")

    # Method to return JSON output
    def to_json(self):
        return self.dict()  # Convert the Pydantic model to a JSON-friendly dictionary

# Initialize the Azure OpenAI client
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")
# --- Initialize the LLM ---
def init_llm():
    """
    Initialize the Azure OpenAI LLM using environment variables.
    """
    return AzureChatOpenAI(
        model="gpt-4",  # Use the correct model version that is available
        openai_api_key=api_key,  # Replace with your actual API key
        azure_endpoint=azure_endpoint,
        api_version="2024-02-01",
        temperature=0.7  # Adjust as needed
    )

In [106]:
# --- Create the prompt for ranking indicators ---
def create_ranking_prompt():
    """
    Create a system and user prompt for ranking competency indicators.
    """
    system_prompt = """
    You are an expert at ranking relevant competency indicators based on the user's tasks and responsibilities.
    """
    user_prompt = """
    The user has the following tasks and responsibilities:
    {tasks}

    Below is a list of competency indicators. Rank these indicators based on how relevant they are for performing the user's tasks and responsibilities. 

    Indicators:
    {indicators}
    """

    return ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("user", user_prompt)
        ]
    )

# --- Create the ranking chain ---
def create_ranking_chain(llm):
    """
    Create the LangChain-style ranking chain using the prompt and structured LLM.
    """
    prompt = create_ranking_prompt()
    structured_llm = llm.with_structured_output(RankedIndicatorsResponse)  # Enforcing output format
    return prompt | structured_llm

# --- Main function to rank indicators ---
def rank_indicators_with_llm(tasks, indicators):
    """
    Use the LLM to rank indicators based on user tasks and return the structured JSON.
    
    :param tasks: User's tasks and responsibilities.
    :param indicators: List of indicators to rank.
    :return: Ranked indicators as structured JSON.
    """
    llm = init_llm()  # Initialize the LLM
    ranking_chain = create_ranking_chain(llm)  # Create the ranking chain
    
    # Prepare the inputs for the chain
    inputs = {
        "tasks": tasks,
        "indicators": ', '.join(indicators)  # Convert list to string for the prompt
    }

    # Run the ranking chain
    ranked_result = ranking_chain.invoke(inputs)
    
    # Return the structured response in JSON format
    return ranked_result.to_json()

In [111]:
import json

# Main function to process user tasks and competencies
def process_user_tasks_and_competencies(tasks, competencies):
    """
    Main function to process user tasks and competencies, and rank indicators.
    
    :param tasks: User's tasks and responsibilities as a string.
    :param competencies: List of competency IDs to query.
    :return: Dictionary of competency IDs with their levels and ranked indicators.
    """
    # Fetch indicators from PostgreSQL (Stub function, replace with real DB fetch)
    competency_data = get_competency_indicators(competencies)

    if competency_data is None:
        print("Failed to retrieve competency data.")
        return None

    # Initialize a dictionary to hold the final structured result
    final_output = {}

    # Iterate through competencies and levels
    for competency_id, levels in competency_data.items():
        # Create a dictionary entry for each competency_id
        final_output[competency_id] = {"competency_id": competency_id, "competency_levels": []}

        for level, indicators in levels.items():
            print(f"Processing competency level: {level} with indicators: {indicators}")

            # Rank the indicators using the previously defined Azure OpenAI GPT function
            ranked_json = rank_indicators_with_llm(tasks, indicators)
            
            # Check if the ranked_json structure is as expected
            if ranked_json and "ranked_indicators" in ranked_json:
                print(f"Ranked JSON received for: {level}")
                
                # Append the ranked results for this level to the competency entry
                final_output[competency_id]["competency_levels"].append({
                    "level": level,
                    "ranked_indicators": ranked_json["ranked_indicators"]  # Use the ranked indicators directly
                })
            else:
                print(f"Error: Unexpected response structure from LLM: {ranked_json}")
                final_output[competency_id]["competency_levels"].append({
                    "level": level,
                    "ranked_indicators": []  # Handle cases where ranking fails
                })

    return final_output



In [112]:
# Example usage
if __name__ == "__main__":
    # User's tasks and responsibilities
    user_tasks = """
    Responsible for leading the design and architecture of complex systems, ensuring that they meet functional and non-functional requirements. 
    Provide technical leadership to development teams and ensure that best practices are followed.
    """

    # List of competency IDs to query (you would replace this with real competency IDs)
    competency_list = [1]  # Example competency IDs

    # Process the user tasks and competencies
    output = process_user_tasks_and_competencies(user_tasks, competency_list)

    # Convert the output to a JSON string for printing
    output_json = json.dumps(output, indent=4)

    # Print the JSON formatted output
    #print(output_json)

Processing competency level: Awareness with indicators: ['Explains what “systems thinking” is and explains why it is important. ', 'Explains what “emergence” is, why it is important, and how it can be “positive” or “negative”in its effect upon the system as a whole.', 'Explains what a “system hierarchy” is and why it is important.', 'Explains what “system context”is for a given system of interest and describes why it is important.', 'Explains why it is important to be able to identify and understand what interfaces are.', 'Explains why it is important to recognise interactions among systems and their elements.', 'Explains why it is important to understand purpose and functionality of a system of interest.', 'Explains how business, enterprise, and technology can each influence the definition and development of the system and vice versa. ', 'Explains why it may be necessary to approach systems thinking in different ways, depending on the situation, and provides examples.']
Ranked JSON re

In [113]:
print(output_json)

{
    "1": {
        "competency_id": 1,
        "competency_levels": [
            {
                "level": "Awareness",
                "ranked_indicators": [
                    {
                        "indicator": "Explains why it is important to understand purpose and functionality of a system of interest.",
                        "rank": 1
                    },
                    {
                        "indicator": "Explains how business, enterprise, and technology can each influence the definition and development of the system and vice versa.",
                        "rank": 2
                    },
                    {
                        "indicator": "Explains what \u201csystems thinking\u201d is and explains why it is important.",
                        "rank": 3
                    },
                    {
                        "indicator": "Explains what \u201csystem context\u201dis for a given system of interest and describes why it is important.",
     